# Storing data

Activity data collected during an experiment is usually represented by an array of
records with the following data:

* time of measurement;

* measured activity value;

* time of day marker (night or day).

We recommend [InfluxDB](https://www.influxdata.com/products/influxdb/) to store activity data.

Each searies of measurements should be represented by a separate InfluxDB table with the following **fields**:

* ``value`` -- numeric values obtained by sensors;

* ``is_night`` -- day/night marker (``True`` for nighttime measurements and ``False`` for daytime measurements).

Other data (like ``sensor_id`` in case of multiple sensors) can also be included in
the table as **tags**.

<div class="alert alert-info">    
In InfluxDB, data can be stored as a field or tag.
Tags are more efficient to construct WHERE-queries but they can hold only string data.
So it's better to have ``value`` and ``is_night`` stored as fields and other data
(required for filtering purpuses) stored as tags.

</div>


For installing and configuring InfluxDB see [the official documentation](https://docs.influxdata.com/influxdb/v2.0/).
Once your InfluxDB is initialized and configured, you are ready to write data to it.

## Writing data to InfluxDB using Python

In the following example, random measurements are generated and written to database via `InfluxDBClient`
which can be downloaded using pip:

```shell
$ pip install influxdb
```

In [1]:
from influxdb import InfluxDBClient
import json
from datetime import datetime, timedelta
import random

In [2]:
# Create a client to access the database.
client = InfluxDBClient('localhost', 8086, 'admin', 'pass', database='my_db')

In [3]:
# Generate random measurements.
measurements=[]
for i in range(5):
    measurements.append({
        'measurement': 'test_series',
        'tags': {
            'sensor_id': '1',
        },
        'time': datetime.fromisoformat('2021-01-01') + timedelta(hours=i),
        'fields': {
            'value': random.uniform(1, 10),
            'is_night': True,
        }
    })
measurements

[{'measurement': 'test_series',
  'tags': {'sensor_id': '1'},
  'time': datetime.datetime(2021, 1, 1, 0, 0),
  'fields': {'value': 2.636775703935856, 'is_night': True}},
 {'measurement': 'test_series',
  'tags': {'sensor_id': '1'},
  'time': datetime.datetime(2021, 1, 1, 1, 0),
  'fields': {'value': 9.634964674586048, 'is_night': True}},
 {'measurement': 'test_series',
  'tags': {'sensor_id': '1'},
  'time': datetime.datetime(2021, 1, 1, 2, 0),
  'fields': {'value': 5.488104584244404, 'is_night': True}},
 {'measurement': 'test_series',
  'tags': {'sensor_id': '1'},
  'time': datetime.datetime(2021, 1, 1, 3, 0),
  'fields': {'value': 2.3051122236289237, 'is_night': True}},
 {'measurement': 'test_series',
  'tags': {'sensor_id': '1'},
  'time': datetime.datetime(2021, 1, 1, 4, 0),
  'fields': {'value': 5.834270867285628, 'is_night': True}}]

In [4]:
# Insert generated data to the database.
client.write_points(measurements);

In [5]:
# Show the inserted data.
client.query('''SELECT (*) FROM "test_series"''')

ResultSet({'('test_series', None)': [{'time': '2021-01-01T00:00:00Z', 'is_night': True, 'sensor_id': '1', 'value': 2.636775703935856}, {'time': '2021-01-01T01:00:00Z', 'is_night': True, 'sensor_id': '1', 'value': 9.634964674586048}, {'time': '2021-01-01T02:00:00Z', 'is_night': True, 'sensor_id': '1', 'value': 5.488104584244404}, {'time': '2021-01-01T03:00:00Z', 'is_night': True, 'sensor_id': '1', 'value': 2.3051122236289237}, {'time': '2021-01-01T04:00:00Z', 'is_night': True, 'sensor_id': '1', 'value': 5.834270867285628}]})